### Use venv(Python 3.11.5) as virtual environment here

In [3]:
import sys
print(sys.executable)
!pip show tensorflow


e:\GenAI\venv\Scripts\python.exe
Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: E:\GenAI\venv\Lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split # Imports the function used to split your dataset into: training set and test set
from sklearn.preprocessing import StandardScaler, LabelEncoder
# LabelEncoder - Converts categorical labels (e.g., "yes", "no") into numeric form (e.g., 1, 0).
# StandardScaler - Many ML algorithms treat bigger numbers as more important, which is unfair here — the unit size is causing bias.
# It standardizes all features to make them similar in scale.
import pickle

In [5]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# DATA PREPROCESSING
# drop irrelevant columns

data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) # axis = 1 means remove these columns
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# ENCODING CATEGORICAL VARIABLES

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender']) 
#fit() → Learn the mapping (e.g., 'Male' → 1, 'Female' → 0)
#transform() → Apply that mapping to the data.
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
# USING ONE HOT ENCODER, ENCODE GEOGRAPHICAL LOCATIONS

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
# data[['Geography']] is used instead of data['Geography'] to keep it as a 2D array, which OneHotEncoder expects.
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [9]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
onehot_encoder_geo.get_feature_names_out(['Geography']) #This tells you the names of the new columns created

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

# Why geo_encoder.toarray()?
# Sparse matrices can’t be directly converted into a pandas DataFrame for viewing or exporting.
# So .toarray() gives you a regular NumPy array


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
# COMBINE ONE HOT ENCODER COLUMNS WITH ORIGINAL DATASET

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()
#data = pd.concat([data, geo_encoded_df], axis=1) - Adds the new one-hot encoded columns (geo_encoded_df) to your existing data DataFrame.

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
# SAVE THE ENCODERS AND SCALARS

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open("onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
# DIVIDING THE DATA INTO DEPENDENT AND INDEPENDENT FEATURES
X = data.drop('Exited',axis=1) # X contains all the input features
Y = data['Exited'] # Y is the target/label, i.e., the actual values of 'Exited'.


# SPLIT THE DATA INTO TRAINING AND TESTING SETS
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42) 
# X_train, Y_train: data used to train the model (80%)
# X_test, Y_test: data used to evaluate how good the model is (20%)
# random_state=42 : You're asking Python to randomly split your dataset into training and test sets
# But because it's random, every time you run this line again, you might get a different split
# So the train and test sets stay the same every time, even if you rerun the code.


# SCALE THESE FEATURES
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # Never use .fit on test data.
# Some features (like balance) may have large values (e.g. 10000) and others small (e.g. age = 35). This can confuse many ML models.
# So we standardize them AND make all features have : Mean=0 & Standard Deviation=1

In [15]:
import pickle

with open("X_train.pkl", "wb") as f:
    pickle.dump(X_train, f)

with open("Y_train.pkl", "wb") as f:
    pickle.dump(Y_train, f)

with open("X_test.pkl", "wb") as f:
    pickle.dump(X_test, f)

with open("Y_test.pkl", "wb") as f:
    pickle.dump(Y_test, f)


In [16]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [17]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [18]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


# ANN IMPLEMENTATION

#### USE tf_env (Python 3.10.18) as virtual environment here

###### 1. Open Anaconda Prompt
conda activate tf_env                     # (or create with: conda create -n tf_env python=3.10)

###### 2. Install TensorFlow and Jupyter kernel
pip install tensorflow
pip install ipykernel

###### 3. Register the environment for Jupyter
python -m ipykernel install --user --name=tf_env --display-name "Python (TF Clean)"


In [19]:
import sys
print(sys.executable)

e:\GenAI\venv\Scripts\python.exe


In [20]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [22]:
X_train.shape[1]

12

In [23]:
# BUILD OUR MODEL

# This model learns to take 12 features from your input data and:

# Process them through 2 hidden layers
# Predict a probability (e.g., 0.87 = 87% chance that label = 1)


model = Sequential([
    Dense(64,activation='relu',input_shape=(12,)), # Hidden Layer 1
    # Dense means a fully connected layer (every neuron is connected to every input).
    # 64 neurons in this layer.
    # activation='relu': Applies the ReLU function (Rectified Linear Unit) — sets all negative values to 0. Helps with non-linearity.
    # input_shape=(12,): You're feeding 12 input features (columns in your dataset). This is only needed in the first layer.
    Dense(32,activation='relu'), # Hidden Layer 2
    # You don’t need to specify input_shape here — it’s inferred from the previous layer.
    Dense(1,activation='sigmoid') # Output Layer
    # 1 neuron: Because you're doing binary classification (e.g., yes/no, 0/1).
]
)


e:\GenAI\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01) # I want to use Adam optimizer to update weights
# learning_rate=0.01 means it makes relatively bigger changes to the model in each step (can be risky if too high).
loss = tensorflow.keras.losses.BinaryCrossentropy() # Use Binary Crossentropy to calculate how wrong the model is
# This is perfect for binary classification (0 or 1 output).
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000024D0287AB60>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [26]:
# COMPILE THE MODEL

model.compile(optimizer = opt,loss = "binary_crossentropy",metrics = ["accuracy"])
# We’re saying:
# “Hey model, when you train:

# Use this optimizer I made (opt)
# Use this loss function (binary_crossentropy)
# Track how accurate you are (accuracy)”

In [27]:
# Set Up the TensorBoard

# You're setting up TensorBoard to visualize how your model trains (loss curves, accuracy, weight histograms, etc)
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# EarlyStopping: Stops training if the model stops improving.
# TensorBoard: Lets you visualize training stats in the browser (curves, histograms, etc.).

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# You’re creating a folder name like:
# "logs/fit20250709-062351" (current date-time style).
# Why? So each training run has its own separate logs.
# datetime.datetime.now() -	Gets the current date and time from your system.
# strftime("%Y%m%d-%H%M%S") - Converts the datetime into a string format like 20250709-094212

tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
# It logs data to that folder every epoch.
# Epoch = One full pass through the entire training dataset e.g. Imagine you have a dataset with 1,000 images, and you train your model on all 1,000 once — that's 1 epoch.
# histogram_freq=1 means: Save weight histograms every 1 epoch

In [28]:
# SET UP EARLY STOPPING

early_stopping_callback = EarlyStopping(monitor = "val_loss", patience=5, restore_best_weights=True)
# monitor - We're telling it to watch the validation loss after each epoch.
# patience - Give the model 5 chances (epochs) to improve.\
# restore_best_weights - After stopping, go back to the model state where validation loss was lowest (i.e., best performance on unseen data).


In [29]:
import pickle
import pandas

with open("X_train.pkl", "rb") as f:
    X_train = pickle.load(f)

with open("Y_train.pkl", "rb") as f:
    Y_train = pickle.load(f)

with open("X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

with open("Y_test.pkl", "rb") as f:
    Y_test = pickle.load(f)


In [30]:
# TRAIN THE MODEL
# This starts the training process. Your model adjusts its weights based on X_train and Y_train.

history = model.fit(
    X_train, Y_train, validation_data = (X_test, Y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback] #
)

# model.fit() - This is the function that trains your model. It feeds data into the model and updates its internal weights to improve predictions.
# validation_data - This is extra data the model doesn't train on, but it tests on after each epoch to see how well it's doing.
# epochs - The number of times the model will go through all the training data. Think of this like 100 rounds of learning.
# history = The result of training (accuracy, loss, etc. for each epoch) is stored in history so you can view or plot it later.

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8093 - loss: 0.4371 - val_accuracy: 0.8540 - val_loss: 0.3471
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8594 - loss: 0.3435 - val_accuracy: 0.8630 - val_loss: 0.3452
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8623 - loss: 0.3344 - val_accuracy: 0.8600 - val_loss: 0.3441
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8592 - loss: 0.3387 - val_accuracy: 0.8590 - val_loss: 0.3458
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8624 - loss: 0.3433 - val_accuracy: 0.8550 - val_loss: 0.3406
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8635 - loss: 0.3286 - val_accuracy: 0.8605 - val_loss: 0.3412
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8602 - loss: 0.3312 - val_accuracy: 0.8620 - val_loss: 0.3405
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8647 - loss: 0.3253 - val_accu

In [31]:
model.save('model.h5')

In [32]:
%load_ext tensorboard

In [33]:
%tensorboard --logdir logs/fit20250710-155806

# you will get output at http://localhost:6006 because tensorflow by default runs at port 6006


Reusing TensorBoard on port 6006 (pid 22424), started 16:48:50 ago. (Use '!kill 22424' to kill it.)

In [34]:
import sys, os
print("Python executable:", sys.executable)
print("sys.prefix (venv root):", sys.prefix)


Python executable: e:\GenAI\venv\Scripts\python.exe
sys.prefix (venv root): e:\GenAI\venv
